In [1]:
import pathlib

In [2]:
image_path = pathlib.Path("images")
train_dir = image_path / "train"
test_dir = image_path / "test"

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(512, 512)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [104]:
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform, target_transform=None)

In [105]:
class_names = train_data.classes
class_dict = train_data.class_to_idx

In [106]:
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [123]:
from torch import nn
class CNN_Model0(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*125*125,
                      out_features=output_shape)
        )
    def forward(self, x):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

In [121]:
model_0 = CNN_Model0(input_shape=3, hidden_units=12, output_shape=len(class_names))

In [109]:
from tqdm.auto import tqdm
from timeit import default_timer as timer

In [110]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [111]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               scheduler):
    """Performs a training step with model trying to learn on data_loader."""
    train_loss, train_acc = 0, 0
    model.train()

    for batch, (x,y) in enumerate(data_loader):
        y_pred = model(x) # gives raw logits as the output

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        acc = accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        train_acc += acc

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
    
    scheduler.step()

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    return train_loss, train_acc

In [112]:
def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    """Performs a test step with model predicting over data_loader."""
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for x,y in data_loader:
            test_pred = model(x)
            loss = loss_fn(test_pred, y)
            test_loss += loss.item()
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
    return test_loss, test_acc

In [113]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          scheduler,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, accuracy_fn, scheduler)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, accuracy_fn)
        print(f"\nEpoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.2f}% | Test loss: {test_loss:.4f} | Test acc: {test_acc:.2f}%")

In [119]:
from torch.optim.lr_scheduler import ExponentialLR
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.005)
scheduler = ExponentialLR(optimizer, gamma=0.95)

In [122]:
start_time = timer()
train(model_0, train_dataloader, test_dataloader, optimizer, accuracy_fn, scheduler, loss_fn)
end_time = timer()
print(f"Total training time = {end_time-start_time}s")

 20%|██        | 1/5 [02:27<09:49, 147.33s/it]


Epoch: 0 | Train loss: 1.3875 | Train acc: 25.24% | Test loss: 1.3855 | Test acc: 30.70%


 40%|████      | 2/5 [05:00<07:32, 150.98s/it]


Epoch: 1 | Train loss: 1.3877 | Train acc: 24.90% | Test loss: 1.3855 | Test acc: 30.70%


 40%|████      | 2/5 [05:07<07:40, 153.59s/it]


KeyboardInterrupt: 

In [18]:
MODEL_PATH = pathlib.Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = "CNN-Model.pt"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
print("Saving model to : ", MODEL_SAVE_PATH)
torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH)

Saving model to :  models\CNN-Model.pt


In [34]:
model_0.load_state_dict(torch.load(f="models\CNN-Model.pt"))

RuntimeError: Error(s) in loading state_dict for CNN_Model0:
	size mismatch for conv_block_1.0.weight: copying a param with shape torch.Size([12, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 3, 3, 3]).
	size mismatch for conv_block_1.0.bias: copying a param with shape torch.Size([12]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for conv_block_1.2.weight: copying a param with shape torch.Size([12, 12, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 16, 3, 3]).
	size mismatch for conv_block_1.2.bias: copying a param with shape torch.Size([12]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for conv_block_2.0.weight: copying a param with shape torch.Size([12, 12, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 16, 3, 3]).
	size mismatch for conv_block_2.0.bias: copying a param with shape torch.Size([12]) from checkpoint, the shape in current model is